In [ ]:
### INJECTING CUSTOM USER INTERFACE NETWORK OPTIONS

## defining a custom user interface block containing css styling / html buttons & cards + javascript logic to make the network graph interactive for users. this html file is injected into the python file below to override default graph interfaces:
# [ css / html ] establishing the style / aesthetics for a container / holder for the search & freeze buttons that will rest in the top left corner of the canvas
# [ css / html ] establishing the style / aesthetics for a search input box that will rest in the container in the top left corner of the canvas
# [ css / html ] establishing the style / aesthetics for a search (bright pink) & freeze (dark nut brown) button that will rest in the container in the top left corner of the canvas + the style / aesthetics for when a user hovers over / interacts with them
# [ css / html ] establishing the style / aesthetics for the pop-up cards linked to each node. these pop-up cards are hidden by default until a user interacts with their connected node.
# [ css / html ] establishing the style / aesthetics for the visit site button & close button embedded within the pop-up cards linked to each node
# [ js ] instructing the program to wait one second before attaching any scripts to ensure pyvis is fully loaded in
# [ js ] accessing internal pyvis variables + establishing an indicator associated with network activity to determine if the network is in a frozen or unfrozen state (to assist with the functionality of the freeze interface button)
# [ js ] establishing the search feature / button press or enter key logic: if the search string is found within a node label in the network, the graph zooms in, highlights, & centers on the searched collaborator's node. if the search string is not found within a node label in the network, the program returns an error message, "calculating connections ... collaborator not found !". if an empty string (no text in the search bar) is passed to the program, the graph re-centers on the user's screen to fit all nodes on screen.
# [ js ] establishing the freeze / unfreeze button: if the previously created indicator displays that the network is not frozen / unfrozen by the user via the freeze button functionality, simulation physics are enabled. if the previously created indicator displays that the network is frozen by the user via the freeze button functionality, simulation physics are disabled. by default, network physics are enabled until the user freezes them by interacting with the freeze button.
# [ js ] establishing node pop-up cards: determines if the node was clicked on by the user & displays information associated with the node in the form of a pop-up card if the clicking interaction did occur. chooses which information to display based on node type & the information associated with the node (for example, hub nodes hold no site / site information. the program scans information associated with the hub node and determines that it does not hold site / site information, therefore a visit site button is not displayed within the linked pop-up card).
# [ js ] establishing that node pop-up cards must appear near where the user's click occurred on a node: forces pop-up cards to appear near their connceted node. also forces any open pop-up card to dissappear if a user clicks in a space where no node exists.
# [ js ] instructs nodes to increase / multiply in size when hovered over or when a user interacts with them
# [ js ] instructs nodes to shrink / return to their original size when they are no longer being hovered over or when a user is no longer interacting with them

%%writefile network_custom_user_interface.html

<style>

  #mynetwork {

    background-color: transparent !important;
    border: none !important;
    outline: none !important;
    box-shadow: none !important;

  }

  .card {

    background-color: transparent !important;
    border: none !important;
    box-shadow: none !important;

  }

  body {

    background-color: transparent !important;
    margin: 0;
    padding: 0;
    overflow: hidden;

  }

  #ui-controls {

    position: absolute;
    top: 20px;
    left: 20px;
    z-index: 1000;

    background: white;
    padding: 10px;
    border-radius: 8px;
    box-shadow: 0 2px 10px rgba(0, 0, 0, 0.1);

    font-family: "Helvetica Neue";

    display: flex;
    gap: 10px;

    }

  #search_input {

    padding: 5px;
    border: 1px solid #ffe5e5;
    border-radius: 2px;
    outline: none;

    font-size: 14px;

    }

  #network_search_button, #freeze_button {

    background-color: #ff1693;
    color: #fffaf0;
    padding: 8px 15px;
    border: none;
    border-radius: 4px;
    cursor: pointer;

    font-weight: bold;
    font-size: 14px;

    }

  #freeze_button { background-color: #513534; }

  #network_search_button:hover { background-color: #e11383; }

  #freeze_button:hover { background-color: #412B2A; }


  #node_pop_up {

    display: none;
    position: absolute;
    width: 280px;
    z-index: 9999;

    background-color: #fffaf0;
    padding: 15px;
    border: 1px solid #fffaf0;
    border-radius: 8px;
    box-shadow: 0 4px 15px rgba(0, 0, 0, 0.3);

    font-family: "Helvetica Neue";

    }

  #node_pop_up h2 {

    margin: 0 0 5px 0;

    color: #ff1693;

    font-size: 15px;

    }

  #node_pop_up p {

    color: #513534;
    margin: 5px 0 10px 0;
    line-height: 1.4;

    font-size: 14px;

    }

  #node_pop_up a.visit-btn {

    display: inline-block;
    margin-top: 5px;

    background-color: #513534;
    color: #fffaf0;
    padding: 8px 15px;
    border-radius: 15px;

    font-weight: bold;
    font-size: 12px;
    text-decoration: none;

  }

  #node_pop_up a.visit-btn:hover { background-color: #412B2A; }

  #close-btn {

    position: absolute;
    top: 5px;
    right: 10px;

    color: #ffe5e5;
    cursor: pointer;

    font-size: 18px;

    }

</style>

<div id = "ui-controls">

  <input type = "text" id = "search_input" placeholder = "find a collaborator ...">
  <button id = "network_search_button">search</button>
  <button id = "freeze_button">freeze</button>

</div>

<div id = "node_pop_up">

  <div id = "close-btn" onclick = "document.getElementById('node_pop_up').style.display='none'">×</div>
  <h2 id = "node_pop_up_name">Name</h2>
  <p id = "node_pop_up_description">Description</p> <a id = "node_pop_up_link" href = "#" target = "_blank" class = "visit-btn">visit site</a>

</div>

<script type = "text/javascript">

  window.onload = function(){

    setTimeout( function(){

      var graph_viewing_window = document.getElementById('mynetwork');

      if (graph_viewing_window) {

        graph_viewing_window.style.border = "none";
        graph_viewing_window.style.backgroundColor = "transparent";

      }

      var network = window.network;
      var nodes = window.nodes;
      var network_frozen = false;

      document.getElementById( "network_search_button" ).onclick = function(){

        var user_search_query = document.getElementById( "search_input" ).value.toLowerCase();

          if ( !user_search_query ){

            network.fit( {

              animation: { duration: 1000, easingFunction: 'easeInOutQuad' }

              } );

            return;

          }

        var all_network_nodes = nodes.get();

          var user_search_node_match = all_network_nodes.find( n => n.label.toLowerCase().includes(user_search_query) );

          if ( user_search_node_match ){

            network.focus( user_search_node_match.id, { scale: 1.2, animation: { duration: 1000, easingFunction: 'easeInOutQuad' } } );
            network.selectNodes( [user_search_node_match.id] );

          } else { alert( "calculating connections ... collaborator not found !" ); }

        };

        document.getElementById( "search_input" ).addEventListener( "keypress" , function( event ){

          if ( event.key === "Enter" ) document.getElementById( "network_search_button" ).click();

        });

        document.getElementById( "freeze_button" ).onclick = function(){

            if ( network_frozen ){

              network.setOptions( { physics: { enabled: true } } );
              document.getElementById( "freeze_button" ).innerText = "freeze";

              network_frozen = false;

            } else {

              network.setOptions({ physics: { enabled: false } } );
              document.getElementById( "freeze_button" ).innerText = "unfreeze";

              network_frozen = true;

            }

          };

          var pop_up = document.getElementById( "node_pop_up" );

          var pop_up_name = document.getElementById( "node_pop_up_name" );
          var pop_up_description = document.getElementById( "node_pop_up_description" );
          var pop_up_link = document.getElementById( "node_pop_up_link" );

          network.on( "click", function ( params ){

            if ( params.nodes.length > 0 ){

                    var node_identifier = params.nodes[0];
                    var node = nodes.get( node_identifier );

                    if ( node.node_name ){

                      pop_up_name.innerText = node.node_name;

                      if ( node.node_description ){

                        pop_up_description.innerText = node.node_description;
                        pop_up_description.style.display = "block";

                        } else {

                          pop_up_description.style.display = "none";

                        }

                      if ( node.node_site ){
                        pop_up_link.href = node.node_site;
                        pop_up_link.style.display = "inline-block";

                        } else {

                            pop_up_link.style.display = "none";

                        }

                      var x_click = params.pointer.DOM.x;
                      var y_click = params.pointer.DOM.y;

                      pop_up.style.left = ( x_click + 15 ) + "px";
                      pop_up.style.top = ( y_click - 20 ) + "px";

                      pop_up.style.display = "block";

                    }

              } else { pop_up.style.display = "none"; }

          });

          network.on( "hoverNode", function ( params ){

            var node = nodes.get( params.node );

            if ( node.base_size ) nodes.update( { id: params.node, size: node.base_size * 1.6 } );

          });

          network.on( "blurNode" , function ( params ){

            var node = nodes.get( params.node );

            if ( node.base_size ) nodes.update( { id: params.node, size: node.base_size } );

          });

    } , 1000 );

  };

</script>

</body>

In [ ]:
### LIBRARIES / SETUP

## installing necessary libraries & dependencies:
# importing libraries for graph creation / structure & interactive graph visualization tools
!pip install pyvis networkx

# importing class to build graph visuals
from pyvis.network import Network
# importing graph dependencies
import networkx as nx

### AESTHETICS / DESIGN

## setting global hub node color configurations:
global_hub_aesthetics = {

    # setting hub node color & border (bright pink + white)
    "background": "#ff1693",
    "border": "#ffffff",
    # setting hub node color & border for when a node is highlighted / interacted with
    "highlight": {

        "background": "#ff1693",
        "border": "#ffffff"

        },
    # setting hub node color & border for when a node is hovered over / interacted with
    "hover": {

        "background": "#ff1693",
        "border": "#ffffff"

        }

}

## setting global collaborator node color configurations:
global_collaborator_aesthetics = {

    # setting collaborator node color & border (dark nut brown + white)
    "background": "#513534",
    "border": "#ffffff",
    # setting collaborator node color & border for when a node is highlighted / interacted with
    "highlight": {

        "background": "#513534",
        "border": "#ffffff"

        },
    # setting collaborator node color & border for when a node is hovered over / interacted with
    "hover": {

        "background": "#513534",
        "border": "#ffffff"

        }

}

### DATA / NODE INFORMATION

## creating dictionaries to map nodes to their relevant context:
# creating a dictionary holding all structural hubs (keys = collaborator institutions / agencies) & related context specific to me (values = degrees / affiliation)
hub_information = {

    "university of missouri": "phd — cultural anthropology (2026) & ma — anthropology (2023)",
    "william & mary": "bs — psychology (honors) & biology",
    "university of california, los angeles": "collaborators"

}

# creating a list of dictionaries, each holding a collaborator & related context (affiliated structural hub from above dictionary + departmental / topical affiliation + site)
collaborator_information = [

    { "name": "karthik panchanathan", "hub": "university of missouri", "dept": "socio-cultural anthropology", "site": "https://www.karthikpanchanathan.com/" },

    { "name": "hannah rubin", "hub": "university of missouri", "dept": "philosophy & metascience", "site": "https://www.hannahrubin.net/" },

    { "name": "rob walker", "hub": "university of missouri", "dept": "socio-cultural anthropology", "site": "https://anthropology.missouri.edu/people/walker" },

    { "name": "greg blomquist", "hub": "university of missouri", "dept": "biological anthropology", "site": "https://gregblomquist.gitlab.io/" },

    { "name": "libby cowgill", "hub": "university of missouri", "dept": "biological anthropology", "site": "https://libbycowgill.phd.sh/" },

    { "name": "lisa sattenspiel", "hub": "university of missouri", "dept": "biological anthropology", "site": "https://sites.google.com/umsystem.edu/sattenspiell" },

    { "name": "mark hannink", "hub": "university of missouri", "dept": "biochemistry", "site": "https://scholar.google.com/citations?user=x7qTItgAAAAJ&hl=en" },

    { "name": "peter cornish", "hub": "university of missouri", "dept": "biochemistry", "site": "https://cafnr.missouri.edu/directory/peter-cornish/" },

    { "name": "joanna schug", "hub": "william & mary", "dept": "socio-cultural psychology", "site": "https://www.wm.edu/as/psych-sciences/facultydirectory/schug_j.php" },

    { "name": "joshua puzey", "hub": "william & mary", "dept": "evolutionary & ecological biology", "site": "https://sites.google.com/view/puzey-lab/home" },

    { "name": "m. drew lamar", "hub": "william & mary", "dept": "computational biology", "site": "https://www.mdlama.me/" },

    { "name": "lee kirkpatrick", "hub": "william & mary", "dept": "evolutionary psychology", "site": "https://www.wm.edu/as/psych-sciences/facultydirectory/retired-facultydirectory/kirkpatrick_l.php" },

    { "name": "helen murphy", "hub": "william & mary", "dept": "evolutionary biology", "site": "https://www.helenmurphy.net/" },

    { "name": "pamela hunt", "hub": "william & mary", "dept": "psychobiology", "site": "https://www.wm.edu/as/psych-sciences/facultydirectory/hunt_p.php" },

    { "name": "sean prall", "hub": "university of california, los angeles", "dept": "socio-cultural anthropology", "site": "https://sprall.github.io/" }

    ]

# for iterative ease, creating a simple list of all hubs
hubs_list = [

    "university of missouri",
    "william & mary",
    "university of california, los angeles"

    ]

### DYNAMIC HUB NODE SIZING SETUP

## on the interactive graph, each hub node size is proportional to the size of collaborators in the overall network associated with that hub:
# initializing an empty dictionary to store each hub's collaborator counts based on dictionaries created above
hub_count = {}

# iterate through each collaborator established in collaborator_information (collaborator list of dictionaries) created above
for node in collaborator_information:

  # if the extracted hub has already been sampled / seen in the loop, the count associated with the hub is increased by one
  if node[ "hub" ] in hub_count:
    hub_count[ node[ "hub" ] ] += 1

  # if the extracted hub has not already been sampled / seen in the loop, it is recorded and its count is set to one
  else:
    hub_count[ node[ "hub" ] ] = 1

### NETWORK GRAPH INITIALIZATION

## creating the main network object:
# establishing the network object
angelas_network = Network(

    # fixing viewing window height
    height = "800px",

    # establishing container usage width
    width = "100%",

    # establishing transparent background
    bgcolor = "rgba(0, 0, 0, 0)",

    # establishing dark nut brown font color
    font_color = "#513534",

    # forcing remote access to relevant javascript libraries to lighten file weight
    cdn_resources = "remote"

    )

### ESTABLISHING HUB NODES

## incorporating hub nodes with dynamic size properties within the network graph:
# looping through established hubs list (hubs_list) to add them into the graph
for hub in hubs_list:

  # retrieves relevant context / descriptive text from hub node dictionary for each hub (reverts to the hubs name if no description given in dictionary)
  affiliation_text = hub_information.get( hub, "institution" )

  # implementing dynamic node sizing for hubs: base size of twenty-five + three pixels per collaborator associated with hub
  hub_dynamic_size = 25 + ( hub_count.get( hub, 1 ) * 3 )

  # adding hub nodes to the network graph
  angelas_network.add_node(

      # unique hub node
      hub,

      # name text label for node hub
      label = hub,

      # applying global node color configurations to hub nodes (bright pink)
      color = global_hub_aesthetics,

      # applying calculated dynamic sizing to each hub node
      size = hub_dynamic_size,

      # forcing base sizing for each hub node (same calculated dynamic sizing as above) to coerce node back to original sizing after a user stops hovering over / interacting with the node
      base_size = hub_dynamic_size,

      # applying font preferences for node text
      font = {"face": "Helvetica Neue", "size": 25, "color": "#513534"},

      # applying thickness of node borders (white)
      borderWidth = 2,

      # applying hidden metadata / properties for each hub's pop-up card (when a hub node is clicked on by a user, a pop-up card appears displaying relevant name & affiliation information established in the original hub dictionary)
      # establishing node type
      node_type = "hub",

      # applying node pop-up title text
      node_name = hub,

      # applying node pop-up description text
      node_description = affiliation_text,

      # setting up application for node pop-up site text if it is added in the future
      node_site = ""

    )

### ESTABLISHING COLLABORATOR NODES

## incorporating collaborator nodes within the network graph:
# looping through established collaborators (node_data) to add them into the graph
for node in collaborator_information:

  # adding collaborator nodes to the network graph
  angelas_network.add_node(

      # unique collaborator node
      node[ "name" ],

      # name text label for collaborator node
      label = node[ "name" ],

      # applying global node color configurations to collaborator nodes (dark nut brown)
      color = global_collaborator_aesthetics,

      # applying set sizing to each collaborator node, smaller than hubs
      size = 18,

      # forcing base sizing for each collaborator node (same set sizing as above, smaller than hubs) to coerce node back to original sizing after a user stops hovering over / interacting with the node
      base_size = 18,

      # applying font preferences for node text
      font = {

          "face": "Helvetica Neue",
          "size": 22,
          "color": "#513534"

          },

      # applying thickness of node borders (white), smaller than hubs
      borderWidth = 1,

      # applying hidden metadata / properties for each collaborator's pop-up card (when a collaborator node is clicked on by a user, a pop-up card appears displaying relevant name & affiliation information established in the original collaborator dictionary)
      # establishing node type
      node_type = " node ",

      # applying node pop-up title text
      node_name = node[ "name" ],

      # applying node pop-up description text
      node_description = node[ "dept" ],

      # applying node pop-up description site
      node_site = node[ "site" ]

    )

### ESTABLISHING NETWORK EDGES

## manually defining the central triangular connection between hubs:
# creating the central structure of the network graph, defined by hubs
angelas_network.add_edge( hubs_list[0], hubs_list[1], color = "#ffb6c1", width = 3, length = 350 )
angelas_network.add_edge( hubs_list[1], hubs_list[2], color = "#ffb6c1", width = 3, length = 350 )
angelas_network.add_edge( hubs_list[2], hubs_list[0], color = "#ffb6c1", width = 3, length = 350 )

# looping through collaborator data to connect each with their associated hub within the network
for node in collaborator_information:

  # creating an edge between each unique collaborator and their associated, unique hub
  angelas_network.add_edge( node[ "hub" ], node[ "name" ], color = "#948484" )

### CONFIGURING PHYSICS OF THE NETWORK

## configuring the physics engine controlling between-node interactions / node movement within the overall network:
# passing json strings to override & establish network physics -
# gravitationalConstant establishes a strong repulsive force to push nodes in the network apart
# centralGravity pulls the nodes of the network towards the center of the canvas / screen
# damping mirrors air friction, instructing the network to stop moving within space after a certain time
# avoidOverlap prevents nodes from overlapping / covering each other in the network visual
# minVelocity instructs the network to stop moving once nodes fall below a certain, slow speed
# interaction allows users to hover over / interact with each node by allowing display of popup cards
angelas_network.set_options("""

{

  "physics": {

    "barnesHut": {

      "gravitationalConstant": -1600,
      "centralGravity": 0.25,
      "damping": 0.425,
      "avoidOverlap": 0.01

    },

    "minVelocity": 0.25

  },

  "interaction": { "hover": true }

}

""" )

### INJECTING CUSTOM USER INTERFACE NETWORK OPTIONS

## defining the network file:
# defining the output html network file
output_html_network_file = "angela_vasishta_my_network.html"

# writing the created network graph into an html file
angelas_network.write_html( output_html_network_file )

# accessing the raw html content of the file generated above in read mode
with open(output_html_network_file, "r") as precustom_network_file:
  html_network_content = precustom_network_file.read()

# calling the external interface file containing custom css styling + html buttons & cards + javascript logic to make the network graph interactive for users
external_network_custom_user_interface = "network_custom_user_interface.html"

# accessing the external interface file to establish a custom interface for the graph
with open(external_network_custom_user_interface, "r") as custom_interface_string:
  custom_user_interface_injection = custom_interface_string.read()

## injecting custom user interface preferences:
# identifying the closing </body> tag within the originally created html file + replacing it with the custom user interface string in the external interface file
angela_vasishta_my_network_graph_010526 = html_network_content.replace( "</body>", custom_user_interface_injection )

### FINAL NETWORK FILE OUTPUT

## overwriting the originally created output file with the custom, modified html version
with open( output_html_network_file, "w" ) as final_network_graph:
    final_network_graph.write( angela_vasishta_my_network_graph_010526 )